In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()
spark

## This notebook reformatted the dataset
In the cleaned dataset, each record represents a data point, such as "Revenue". We aimed to pivot the datapoints into columns so that each record would represent the data of a company for a fiscal year.
### read in the cleaned dataset

In [44]:
df = spark.read.parquet('s3://sec-finc/uploaded/')

### drop duplication

In [45]:
df = df.dropDuplicates(['company_name', 'document_fiscal_year_focus','datapoint_name', 'string_value'])

In [46]:
df.show()

+-------+--------------------+------------+--------------------+-----------+--------------------------+------------+--------------------+--------------------+----------+----------+------------+------------+-------------+--------+----------+---------------------+--------------+--------------------------+------------+-------------+------------+---------+
|    cik|        company_name|assigned_sic|accession_number_int|filing_date|document_fiscal_year_focus|datapoint_id|      datapoint_name|     datapoint_label|start_date|  end_date|period_month|string_value|numeric_value|decimals|      unit|parent_datapoint_name|statement_type|report_section_description|     version|segment_label|segment_hash|footnotes|
+-------+--------------------+------------+--------------------+-----------+--------------------------+------------+--------------------+--------------------+----------+----------+------------+------------+-------------+--------+----------+---------------------+--------------+-------------

In [47]:
df.createOrReplaceTempView('SelectedDF')

### only include start_date & end_date is in the fiscal_year + period_month = 12

In [48]:
temp = spark.sql('''
    SELECT *
    FROM SelectedDF
    WHERE (YEAR(start_date) = document_fiscal_year_focus)
    AND (period_month = 12)
''')

In [49]:
temp.createOrReplaceTempView('SelectedDF2')

### pivot the table

In [62]:
from pyspark.sql import functions as F
transDF = temp.groupby(["cik","company_name","assigned_sic",
                        "document_fiscal_year_focus","start_date","end_date"])\
    .pivot("datapoint_name")\
    .agg(F.mean("numeric_value"))

transDF.show()

+-------+--------------------+------------+--------------------------+----------+----------+--------------------------+---------------+--------------+---------------------+-----------------------+-----------------------------+-------------------------------+---------------+---------------------------------+------------------------------------+------------------------------------+-----------------------------+----------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+-------------+-----------------+-------------------+----------------------------------+-------------------+------------------------------+------------------------+--------------------------------------------+---------------------------------------

In [63]:
transDF.count()

41781

### count # of records not null for each column

In [64]:
from pyspark.sql.functions import col, count, isnan, lit, sum
def count_not_null(c, nan_as_null=False):
    """Use conversion between boolean and integer
    - False -> 0
    - True ->  1
    """
    pred = col(c).isNotNull() & (~isnan(c) if nan_as_null else lit(True))
    return sum(pred.cast("integer")).alias(c)
transDF.agg(*[count_not_null(c) for c in transDF.columns]).show()

+-----+------------+------------+--------------------------+----------+--------+--------------------------+---------------+--------------+---------------------+-----------------------+-----------------------------+-------------------------------+---------------+---------------------------------+------------------------------------+------------------------------------+-----------------------------+----------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+-------------+-----------------+-------------------+----------------------------------+-------------------+------------------------------+------------------------+--------------------------------------------+------------------------------------------+--------

In [65]:
transDF.createOrReplaceTempView('transDF')

In [2]:
### discover bad cases for still containing duplication

In [66]:
spark.sql('''
select 
company_name,
document_fiscal_year_focus,
cnt
from
    (SELECT company_name,
    document_fiscal_year_focus,
    count(*) as cnt
    FROM transDF
    group by company_name,document_fiscal_year_focus) as a
where a.cnt >=2
''').show()

+--------------------+--------------------------+---+
|        company_name|document_fiscal_year_focus|cnt|
+--------------------+--------------------------+---+
|        US FUEL CORP|                      2012|  2|
|HERITAGE-CRYSTAL ...|                      2012|  2|
|UNITED FINANCIAL ...|                      2013|  2|
|CONNECTONE BANCOR...|                      2013|  2|
|XENITH BANKSHARES...|                      2011|  2|
|GRIFFIN CAPITAL E...|                      2017|  2|
|    BBX CAPITAL CORP|                      2013|  2|
|DOLAT VENTURES, INC.|                      2011|  2|
|    BBX CAPITAL CORP|                      2011|  2|
|UNITED FINANCIAL ...|                      2011|  2|
|PANACEA GLOBAL, INC.|                      2013|  2|
|    BBX CAPITAL CORP|                      2012|  2|
|CHART INDUSTRIES INC|                      2018|  2|
| SYNEOS HEALTH, INC.|                      2016|  2|
| CENTRUS ENERGY CORP|                      2018|  2|
|MISSION COMMUNITY...|      

In [69]:
a = spark.sql('''
    SELECT *
    FROM transDF
    WHERE document_fiscal_year_focus=2012
    AND company_name like '%US FUEL%'
''')
a.show()

+-------+------------+------------+--------------------------+----------+----------+--------------------------+---------------+--------------+---------------------+-----------------------+-----------------------------+-------------------------------+---------------+---------------------------------+------------------------------------+------------------------------------+-----------------------------+----------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+-------------+-----------------+-------------------+----------------------------------+-------------------+------------------------------+------------------------+--------------------------------------------+------------------------------------------+----

In [80]:
# pivot the table
from pyspark.sql import functions as F
transDF = temp.groupby(["company_name","assigned_sic",
                        "document_fiscal_year_focus"])\
    .pivot("datapoint_name")\
    .agg(F.mean("numeric_value"))

transDF.show()

+--------------------+------------+--------------------------+--------------------------+---------------+--------------+---------------------+-----------------------+-----------------------------+-------------------------------+---------------+---------------------------------+------------------------------------+------------------------------------+-----------------------------+----------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+-------------+-----------------+-------------------+----------------------------------+-------------------+------------------------------+------------------------+--------------------------------------------+------------------------------------------+------------------------+-

In [81]:
transDF.count()

41741

In [82]:
from pyspark.sql.functions import col, count, isnan, lit, sum
## count # of records not null for each column
def count_not_null(c, nan_as_null=False):
    """Use conversion between boolean and integer
    - False -> 0
    - True ->  1
    """
    pred = col(c).isNotNull() & (~isnan(c) if nan_as_null else lit(True))
    return sum(pred.cast("integer")).alias(c)
transDF.agg(*[count_not_null(c) for c in transDF.columns]).show()

+------------+------------+--------------------------+--------------------------+---------------+--------------+---------------------+-----------------------+-----------------------------+-------------------------------+---------------+---------------------------------+------------------------------------+------------------------------------+-----------------------------+----------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+-------------+-----------------+-------------------+----------------------------------+-------------------+------------------------------+------------------------+--------------------------------------------+------------------------------------------+------------------------+---------

In [83]:
transDF.createOrReplaceTempView('transDF')

In [84]:
spark.sql('''
select 
company_name,
document_fiscal_year_focus,
cnt
from
    (SELECT company_name,
    document_fiscal_year_focus,
    count(*) as cnt
    FROM transDF
    group by company_name,document_fiscal_year_focus) as a
where a.cnt >=2
''').show()

+--------------------+--------------------------+---+
|        company_name|document_fiscal_year_focus|cnt|
+--------------------+--------------------------+---+
|UNITED FINANCIAL ...|                      2013|  2|
|XENITH BANKSHARES...|                      2011|  2|
|UNITED FINANCIAL ...|                      2011|  2|
|XENITH BANKSHARES...|                      2012|  2|
|XENITH BANKSHARES...|                      2013|  2|
|XENITH BANKSHARES...|                      2014|  2|
|SPIRIT REALTY CAP...|                      2012|  2|
|XENITH BANKSHARES...|                      2015|  2|
|UNITED FINANCIAL ...|                      2012|  2|
+--------------------+--------------------------+---+



In [86]:
a = spark.sql('''
    SELECT *
    FROM transDF
    WHERE document_fiscal_year_focus=2013
    AND company_name like '%XENITH BANKSHARES%'
''')
a.show()

+--------------------+------------+--------------------------+--------------------------+---------------+--------------+---------------------+-----------------------+-----------------------------+-------------------------------+---------------+---------------------------------+------------------------------------+------------------------------------+-----------------------------+----------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+------------------------------------------+--------------------------------------------------------------+-------------+-----------------+-------------------+----------------------------------+-------------------+------------------------------+------------------------+--------------------------------------------+------------------------------------------+------------------------+-

Bad cases have different assigned_sic (identify which industry a company belongs) for the same company, this is most likely happened due to one company has two sectors which were assigned to different industries. In addition, there's only 2 cases for around 10,000 companies, so we did not apply further cleaning towards those cases.